# Sprawozdanie z **Obliczeń ewolucyjnych**
## Zadanie 1 - Algorytm *Particle Swarm Optimization*

# Konfiguracja
Przy pomocy klasy Config można wybrać zastosowany algorytm oraz określić jego parametry:    
- w - współczynnik zmęczenia.   
- c1 - współczynnik przyspieszenia w kierunku własnego minimum cząstki
- c2 - współczynnik przyspieszenia w kierunku globalnego minimum cząstek
- iterations - ilość iteracji algorytmu  
- target_error - najniższy stopień błędu - po osiągnięciu tego wyniku algorytm zatrzymuje się  
- n_particles - ilość cząstek  
- arguments_dimensions - liczba wymiarów, w których rozkładane są cząsteczki   
- d_min - minimalna wartość współrzędnej dla każdego z wymiarów   
- d_max - maksymalna wartość współrzędnej dla każdego z wymiarów   
- fitness - wybór funkcji dostosowującej położenie cząstek         
 Dla celów badawczych, w dalszej części zaprezentowane są różne zestawy wartości konfiguracyjnych.

In [17]:
!pip install deap k3d > /dev/null
%matplotlib inline
from pathlib import Path
import json

class Config(object):
    def __init__(self, algorithmType, w, c1, c2, custom_c2, iterations, target_error, n_particles, arguments_dimensions, d_min, d_max, fitness):
        self.algorithmType = algorithmType
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.custom_c2 = custom_c2
        self.iterations = iterations
        self.target_error = target_error
        self.n_particles = n_particles
        self.arguments_dimensions = arguments_dimensions
        self.d_min = d_min
        self.d_max = d_max
        self.d_max = d_max
        self.fitness = fitness

def as_config(dct):
    return Config(
        dct['algorithmType'],
        dct['w'],
        dct['c1'],
        dct['c2'],
        dct['custom_c2'] if dct.get('custom_c2') else False,
        dct['iterations'], 
        dct['target_error'], 
        dct['n_particles'],
        dct['arguments_dimensions'],
        dct['d_min'], 
        dct['d_max'],
        dct['fitness']
        )

# Algorytm genetyczny z użyciem "deap"
W celu porównania samodzielnie zaimplementowanego algorytmu PSO do algorytmów genetycznych, poniżej zaimplementowano funkcję zwracające wyniki działania tego algorytmu z wykorzystaniem biblioteki "deap".  

In [18]:
import random
import numpy
import inspect
from deap import algorithms, base, creator, tools

# translates into tuple
def gaFitness(individual):
    return fitness(individual),

def cxTwoPointCopy(ind1, ind2):
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

def genetic():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", lambda: (cfg.d_max-cfg.d_min)*random.random() + cfg.d_min)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=cfg.arguments_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", gaFitness)
    toolbox.register("mate", cxTwoPointCopy)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    pop = toolbox.population(n=cfg.n_particles)
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=cfg.iterations, stats=stats, halloffame=hof)

    return pop, stats, hof

# Klasy algorytmu PSO
Poniżej przedstawione są klasy repezentujące "rój"
### Particle
Klasa reprezentacja jedną cząsteczkę w przestrzeni poszukiwań:  
- position - pozycja w przestrzeni zdefiniowanej w pliku konfiguracyjnym. 
- pbest_position - najlepsza pozycja cząsteczki
- pbest_value - najlepsza osiągnięta przez tę czastęczkę wartość
- velocity - wektor, który przechowuje informacje nt. tzw prędkości cząsteczki.  
### Space
Klasa reprezentująca przestrzeń poszukiwań, w której poruszają się cząsteczki
- particles[] - tablica cząsteczek
- gbest_position - globalnie najlepsza pozycja cząsteczki - inicjowana losowo 
- gbest_value - najlepsza wartość funkcji przystosowania dowolnej z cząsteczek

In [19]:
import random
import numpy as np

class Particle():
    def __init__(self):
        self.position = (cfg.d_max - cfg.d_min) * np.random.random(cfg.arguments_dimensions) + cfg.d_min
        self.pbest_position = self.position
        self.pbest_value = float('inf')
        self.velocity = np.zeros(cfg.arguments_dimensions)
    
    def move(self):
        self.position = self.position + self.velocity
        for i in range(len(self.position)):
            if self.position[i] > cfg.d_max:
                self.position[i] = cfg.d_max
            if self.position[i] < cfg.d_min:
                self.position[i] = cfg.d_min

    def __str__(self):
        print(f'My position is {self.position}, my pbest is {self.pbest_position}')

class Space():
    def __init__(self):
        self.particles = []
        self.gbest_value = float('inf')
        self.gbest_position = np.array([x * np.random.random_sample()*cfg.d_max for x in range(cfg.arguments_dimensions)])

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()

    def set_pbest(self):
        for particle in self.particles:
            fitness_cadidate:float = fitness(particle.position)
            if(particle.pbest_value > fitness_cadidate):
                particle.pbest_value = fitness_cadidate
                particle.pbest_position = particle.position

    def set_gbest(self):
        for particle in self.particles:
            best_fitness_cadidate = fitness(particle.position)
            if(self.gbest_value > best_fitness_cadidate):
                self.gbest_value = best_fitness_cadidate
                self.gbest_position = particle.position

    def move_particles(self):
        for particle in self.particles:
            new_velocity = (cfg.w*particle.velocity) + (cfg.c1*np.random.random_sample()) * (particle.pbest_position - particle.position) + \
                            (np.random.random_sample()*cfg.c2) * (self.gbest_position - particle.position)
            particle.velocity = new_velocity
            particle.move()


# Wykresy
Wykorzystano bibliotekę k3d w celu przedstawienia trójwymiarowych wykresów "roju", tam gdzie było to możliwe.

In [20]:
from numpy import exp,array,float32
import k3d
from uuid import uuid4

def plot_particles(particles):
    if len(particles) == 0:
        print("No particles to visualize !")
        return
    if len(particles[0].position) != 2:
        print("I can only visualize three dimensional function !")
        return
    x = list(map(lambda p: (p.position[0], p.position[1], fitness(p.position)), particles))
    plot = k3d.plot(name=str(uuid4()))
    plt_points = k3d.points(positions=x, point_size=0.2)
    plot += plt_points
    plt_points.shader='3d'
    plot.display()

# Funkcje przystosowania
Poniższa sekcja przestawia funkcje dostosowania cząsteczek:  
Sphere
![title](pics/sphere.png)
F2
![title](pics/f2.png)
Griewank
![title](pics/griewank.png)   
Rosenbrock
![title](pics/rosen.png)   
Zakharov
![title](pics/zakharov.png)   

In [21]:
from numpy import sum, asarray_chkfinite, arange, prod, sqrt, cos, sin
from functools import reduce
def fitness(position, fr=4000):
    if cfg.fitness == "sincos":
        return sin(position[0]) + cos(position[1])
    if cfg.fitness == "sincos2":
        return sin(position[0]) + cos(position[1]) + position[0] + position[1]
    elif cfg.fitness == "sphere":
        return reduce(lambda p,n: p + n**2, position)
    elif cfg.fitness == "rosen":
        return sum(100.0*(position[1:]-position[:-1]**2.0)**2.0 + (position[:-1]-1)**2.0)
    elif cfg.fitness == "griewank":
        n = len(position)
        j = arange( 1., n+1 )
        s = sum( position**2 )
        p = prod( cos( position / sqrt(j) ))
        return s/fr - p + 1
    elif cfg.fitness == "zakharov":
        x = np.asarray_chkfinite(position)
        n = len(x)
        j = np.arange( 1., n+1 )
        s2 = sum( j * x ) / 2
        return sum( x**2 ) + s2**2 + s2**4
    elif cfg.fitness == "f2":
        s = 0
        for x in range(1, len(position)):
            s = s + (position[x]-x)**2
        return s
    else:
        print("Unknown fitenss function")
        raise

# Pętla aplikacji
Pętla programowa odpowiedzialna za wywoływanie logiki wybranego algorytmu PSO lub GA.

In [22]:
import numpy as np
class App():
    def run(self):
        if(cfg.algorithmType == "pso"):
            c2_begin = cfg.c2
            search_space = Space()
            particles_vector = [Particle() for _ in range(cfg.n_particles)]
            search_space.particles = particles_vector
            plot_particles(search_space.particles)            
#             search_space.print_particles()
            iteration = 0
            while iteration < cfg.iterations:
                if cfg.custom_c2 == True:
                    cfg.c2 = c2_begin * ((cfg.iterations - iteration)/cfg.iterations)
                last_gbest = search_space.gbest_value
                search_space.set_pbest()
                search_space.set_gbest()
                search_space.move_particles()
                iteration += 1
                if abs(last_gbest - search_space.gbest_value) < cfg.target_error:
                    break
            print(f"The best solution is in: {search_space.gbest_position}, fitness function value is: {fitness(search_space.gbest_position)}. Achieved in {iteration} iterations")
            plot_particles(search_space.particles)
#             search_space.print_particles()
        elif(cfg.algorithmType == "ga"):
            genetic()

        else:
            print(f"Unknown algorithm type: {cfg.algorithmType}")

### Ziarno losowości

In [23]:
from numpy import random as nrand
nrand.seed(0)
from random import seed
seed(0)

# Badania
Przeprowadzono eksperyment uruchomień algorytmu dla różnych zestawów parametrów konfiguracyjnych. Poniżej przedstawiono poszczególne przykłady wraz z wynikami w postaci wykresów 3D lub najlepszego osiągniętego wyniku.

### Funkcja sin(x) + cos(y)
Przeprowadzono doświadczenie dla dwóch zestawów parametrów wagi inercyjnej, składników kognitywnego oraz socjalnego.

<table style="width:150px;font-size:20px;text-align:left;" >
  <tr style="text-align:left;">
    <th>W</th>
    <th>C1</th> 
    <th>C2</th>
  </tr>
  <tr>
    <td>0.5</td>
    <td>0.8</td>
    <td>0.8</td>
  </tr>
  <tr>
    <td>0.5</td>
    <td>0.2</td>
    <td>0.4</td>
  </tr>
</table>

In [24]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "sincos",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "custom_c2": true,
    "target_error": 0,
    "iterations": 400,
    "n_particles": 1000,
    "arguments_dimensions": 2,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Output()

The best solution is in: [4.71238898 3.14159266], fitness function value is: -2.0. Achieved in 400 iterations


Output()

In [9]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "sincos",
    "w": 0.5,
    "c1": 0.2,
    "c2": 0.4,
    "custom_c2": true,
    "target_error": 0,
    "iterations": 400,
    "n_particles": 1000,
    "arguments_dimensions": 2,
    "d_min": -10,
    "d_max": 10
}"""ć
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Output()

The best solution is in: [-1.57079631  3.14159264], fitness function value is: -2.0. Achieved in 400 iterations


Output()

In [10]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "sphere",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 0,
    "iterations": 5,
    "n_particles": 2000,
    "arguments_dimensions": 2,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Output()

The best solution is in: [-1.00000000e+01 -5.17245202e-06], fitness function value is: -9.999999999973246. Achieved in 5 iterations


Output()

### Funkcja *zakharov*
Zaimplementowano możliwość zastosowania malejącej liniowo wraz z rosnącym numerem iteracji wartości składnika socjalnego. Użycie tej modyfikacji kontrolowane jest poprzez wartość klucza "custom_c2", odpowiednio na "true" lub "false".  
Zastosowanie tego rozwiązania przetestowano na przykładzie funkcji "Zakharov" dla przedstawionych poniżej parametrów.  
W przeprowadzonym eksperymencie, zastosowanie tej modyfikacji umożliwiło osiągnięcie poprawy wyniku z **12.62 do 10.82**
<table style="width:1000px;font-size:20px;text-align:left;" >
  <tr style="text-align:left;">
    <th>Funkcja</th>
    <th>mss</th>
    <th>num_dim</th>
    <th>range</th>
    <th>W</th>
    <th>C1</th> 
    <th>C2</th>
  </tr>
  <tr style="text-align:left;">
    <th>Zakharov</th>
    <th>Nie</th>
      <th>20</th>
      <th>&lt;-10, 10&gt; </th>
      <th>0.5</th>
      <th>0.7</th>
      <th>0.8</th>
  </tr>
  <tr style="text-align:left;">
    <th>Zakharov</th>
    <th>Tak</th>
      <th>20</th>
      <th>&lt;-10, 10&gt; </th>
      <th>0.5</th>
      <th>0.7</th>
      <th>0.8</th>
  </tr>
</table>
<div>
    <p>mss - Malejący składnik socjalny</p>
    <p>num_dim - Liczba wymiarów przestrzeni poszukiwań</p>
    <p>range - Przedział wartości pojedynczego wymiaru</p>
</div>

In [11]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "zakharov",
    "w": 0.5,
    "c1": 0.7,
    "c2": 0.8,
    "custom_c2": false,
    "target_error": 0,
    "iterations": 100,
    "n_particles": 600,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

I can only visualize three dimensional function !
The best solution is in: [-0.43248185 -0.83503284 -0.6426348   1.22867542  1.25658904  0.20164191
  0.09892838  1.66899168  0.88216377 -0.39351972 -0.91785579  0.155929
 -0.27878025 -0.99105565 -1.1407082   0.07778228 -0.55522984 -0.06637303
  0.58302337  0.70969746], fitness function value is: 12.619413306436881. Achieved in 100 iterations
I can only visualize three dimensional function !


In [12]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "zakharov",
    "w": 0.5,
    "c1": 0.7,
    "c2": 0.8,
    "custom_c2": true,
    "target_error": 0,
    "iterations": 100,
    "n_particles": 600,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

I can only visualize three dimensional function !
The best solution is in: [ 0.60465769 -0.35665909 -0.07423155  0.92442713  1.16126495  0.75378846
 -0.74743498  0.94047621  0.21316706 -0.08039428 -0.02501439 -1.9762709
  0.36800107  0.63614627 -0.44119699 -0.63879784  0.02025505 -0.40833985
  0.91214695 -0.02137138], fitness function value is: 10.818307372839868. Achieved in 100 iterations
I can only visualize three dimensional function !


## "Particle Swarm Optimization" vs "Genetic Algorithm"
Zastosowano algorytmy: PSO oraz GA dla funkcji oznaczonej "f2" dla następujących parametrów:
<table style="width:1000px;font-size:20px;text-align:left;" >
  <tr style="text-align:left;">
    <th>Algorytm</th>
    <th>Funkcja</th>
    <th>Wynik</th>
    <th>num_dim</th>
    <th>range</th>
    <th>particles</th>
    <th>W</th>
    <th>C1</th> 
    <th>C2</th>
  </tr>
  <tr style="text-align:left;">
    <th>PSO</th>
    <th>F2</th>
    <th>342.01</th>
    <th>20</th>
    <th>&lt;-10, 10&gt; </th>
    <th>2000</th>
    <th>0.5</th>
    <th>0.7</th>
    <th>0.8</th>
  </tr>
  <tr style="text-align:left;">
    <th>GA</th>
    <th>F2</th>
    <th>304.431	</th>
    <th>20</th>
    <th>&lt;-10, 10&gt; </th>
    <th>2000</th>
    <th>0.5</th>
    <th>0.7</th>
    <th>0.8</th>
  </tr>
  <tr style="text-align:left;">
    <th>PSO</th>
    <th>Griewank</th>
    <th>1.97</th>
    <th>20</th>
    <th>&lt;-10, 10&gt; </th>
    <th>2000</th>
    <th>0.5</th>
    <th>0.7</th>
    <th>0.8</th>
  </tr>
  <tr style="text-align:left;">
    <th>GA</th>
    <th>Griewank</th>
    <th>0</th>
    <th>20</th>
    <th>&lt;-10, 10&gt; </th>
    <th>2000</th>
    <th>0.5</th>
    <th>0.7</th>
    <th>0.8</th>
  </tr>
</table>
<p>num_dim - Liczba wymiarów przestrzeni poszukiwań
<p>range - Przedział wartości pojedynczego wymiaru</p>

### Funkcja "F2"

In [39]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "f2",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 1e-8,
    "iterations": 80,
    "n_particles": 2000,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

I can only visualize three dimensional function !
The best solution is in: [ 3.87574243  0.93270256  1.97305033  2.95619663 10.          5.03634619
 10.          7.04612528 10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.        ], fitness function value is: 342.01062255300553. Achieved in 80 iterations
I can only visualize three dimensional function !


In [40]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "ga",
    "fitness": "f2",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 0,
    "iterations": 80,
    "n_particles": 2000,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

gen	nevals	avg 	std   	min    	max    
0  	2000  	3111	595.65	1363.65	4843.76
1  	1179  	2585.82	441.435	1276.25	4098.01
2  	1240  	2199.83	349.826	957.29 	3387.78
3  	1170  	1912   	289.636	957.29 	3109.45
4  	1189  	1671.65	244.52 	853.78 	2437.91
5  	1199  	1477.9 	208.568	766.252	2318.89
6  	1230  	1313.45	186.704	722.285	1980.24
7  	1236  	1164.14	172.357	628.071	1861.21
8  	1211  	1040.09	152.4  	577.798	1596.13
9  	1265  	931.88 	134.323	552.661	1580.68
10 	1237  	835.507	116.192	421.509	1475.64
11 	1197  	755.789	105.451	421.509	1398.94
12 	1222  	692.313	98.5374	448.357	1391.39
13 	1210  	637.489	88.682 	432.164	1422.1 
14 	1185  	591.247	82.0734	388.336	1171.69
15 	1209  	550.354	80.3415	368.103	1061.43
16 	1188  	514.411	73.03  	361.209	1048.8 
17 	1206  	488.523	78.435 	368.875	1170.87
18 	1223  	460.898	77.1373	354.47 	1094.45
19 	1187  	435.872	71.0481	350.958	1054.12
20 	1199  	421.923	73.4807	350.245	997.991
21 	1179  	407.676	74.9837	344.057	1000.7 
22 	1166  	394.266	

### Funkcja "Griewank"

In [41]:
seed = 0
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "griewank",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 1e-8,
    "iterations": 80,
    "n_particles": 200,
    "arguments_dimensions": 20,
    "d_min": -600,
    "d_max": 600
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

I can only visualize three dimensional function !
The best solution is in: [-14.90166344  -7.35490992  -5.81904367  14.69154484   3.29545856
   2.30207341 -12.96272031 -17.68649468 -14.02710741  26.19057764
   8.09667497  -9.71331277 -15.72031188 -12.42935725   8.34415879
  -7.92447079  -3.63687155 -14.94507237  -7.76285952 -31.57517501], fitness function value is: 1.973336522415012. Achieved in 80 iterations
I can only visualize three dimensional function !


In [42]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "ga",
    "fitness": "griewank",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 0.1,
    "iterations": 80,
    "n_particles": 2000,
    "arguments_dimensions": 20,
    "d_min": -600,
    "d_max": 600
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

gen	nevals	avg    	std    	min   	max    
0  	2000  	604.569	121.265	271.08	1076.99
1  	1179  	497.878	92.2881	232.41	752.828
2  	1240  	414.359	77.4466	165.712	640.931
3  	1170  	346.996	66.9919	112.17 	593.365
4  	1189  	287.412	57.5134	110.427	498.594
5  	1199  	236.395	50.7673	69.1183	402.97 
6  	1230  	192.593	43.6303	57.3238	371.568
7  	1236  	153.415	36.186 	43.2228	303.947
8  	1211  	121.372	30.0106	37.1446	257.869
9  	1265  	94.7346	24.7287	14.0273	204.144
10 	1237  	72.6977	19.2691	14.0273	151.83 
11 	1197  	55.9686	15.5179	10.8546	108.237
12 	1222  	42.3186	12.4134	8.64222	85.1708
13 	1210  	31.3677	9.3247 	7.15166	71.3821
14 	1185  	23.1473	7.10307	4.11921	47.0039
15 	1209  	16.9177	5.16974	2.96997	40.9941
16 	1188  	12.6044	3.75396	2.84505	31.3731
17 	1206  	9.40968	2.93546	1.81763	21.2634
18 	1223  	6.85662	2.43237	1.53898	17.1339
19 	1187  	4.75847	1.75887	0.216913	11.7121
20 	1199  	3.33699	1.05457	0.655589	10.8538
21 	1179  	2.49021	0.653414	0.193022	5.88514
22 	1166  